In [1]:
import math
import numpy as np
from scipy.optimize import fsolve, approx_fprime
from scipy.stats import entropy

In [2]:
# L : number of glycan types
# N : number of lattice points
# P = [p(1,1),...,p(1,N), ... ,p(L,1),...,p(L,N)]  (LN in row-major order)
# W = [[w11,...,w1N], ... ,[wL1,...,wLL]]          (LxL)

def total_entropy(P, L, N):
    return np.sum([entropy(P[i]) for i in range(L)])

def P_neigh(P, N, t, x):  # probability that type t neighbours location x, with bounds checking on x
    if x == 0:
        return P[t][x+1]
    elif x == N-1:
        return P[t][x-1]
    else:
        return P[t][x-1] + P[t][x+1]

def neighbour_constr_1D(Pt, Lmd, L, N, W):  
    nc = np.array([[np.sum(np.array([P_neigh(Pt,N,t2,i)*Pt[t1][i] for i in range(N)])) 
                    for t2 in range(L)] for t1 in range(L)])
    nc = np.subtract(nc, W)
    
    return nc

def Lagrange_1D(x, L, N, W):
    P, Lm = x[:N*L], x[N*L:]
    Pt = np.array([P[i*N:(i+1)*N] for i in range(L)])
    Lmd = np.array([Lm[i*L:(i+1)*L] for i in range(L)])
    
    lc = np.multiply(Lmd, neighbour_constr_1D(Pt, Lmd, L, N, W))
    
    return -total_entropy(Pt, L, N) + np.sum(lc)
    
def safelog(x):
    if x <= 0:
        return 0
    return math.log(x)
    
def Jacobian_1D(x, L, N, W):
    P, Lm = x[:N*L], x[N*L:]
    Pt = np.array([P[i*N:(i+1)*N] for i in range(L)])
    Lmd = np.array([Lm[i*L:(i+1)*L] for i in range(L)])
    dLdp = np.zeros(L*N)
    dLdLmd = np.zeros(L*L)
    
    for t in range(L):
        for i in range(N):
            #dLdp[t*N + i] -= 1 + safelog(Pt[t][i])  # prevent bounds error 
            dLdp[t*N + i] +=  np.sum(np.array([(Lmd[t][t1]+Lmd[t1][t])*P_neigh(Pt, N, t1, i) for t1 in range(L)]))
    
    dLdLmd = np.ndarray.flatten(neighbour_constr_1D(Pt, Lmd, L, N, W))
            
    return np.append(dLdp, dLdLmd)
      

In [3]:
def solve(x, L, N, W):
    sol = fsolve(Jacobian_1D, x, args=(L, N, W), full_output=True)
    print("Solution\n", sol)
    print("\nX\n", sol[0])
    pn = sol[0][:N*L]
    print("\nPn\n", [pn[i*N:(i+1)*N] for i in range(L)])
    print("\nLagrangian\n", Lagrange_1D(sol[0], L, N, W))
    print("\nTotal Entropy\n", total_entropy(np.array([pn[i*N:(i+1)*N] for i in range(L)]), L, N))
   
L = 2
N = 10
Lmd = np.zeros(L*L)
W = np.array([[0.1, 0.9], [0.8, 0.2]])
P = np.asarray([0.5] * (L*N))
x = np.append(P, Lmd)
print("L = {}\nN = {}\nP = {}\nLmd = {}\nW = {}\nx = {}\n".format(L, N, P, Lmd, W, x))
print(Lagrange_1D(x, L, N, W))
print(Jacobian_1D(x, L, N, W))

solve(x, L, N, W)

L = 2
N = 10
P = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5]
Lmd = [0. 0. 0. 0.]
W = [[0.1 0.9]
 [0.8 0.2]]
x = [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5
 0.5 0.5 0.  0.  0.  0. ]

-4.605170185988091
[0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  4.4 3.6 3.7 4.3]
Solution
 (array([-3.06842405e+00, -1.28528193e+00, -2.48919792e+00, -2.70293072e+00,
       -2.78069424e+00, -1.89358835e+00,  7.63062225e+00, -1.26540364e+00,
        5.34567617e+00,  7.98160809e-01,  2.94053327e-01, -1.66515383e+00,
       -6.05194999e-01,  1.57680662e-01,  9.41157109e-02, -1.61167359e+00,
        3.18307089e+00,  7.14109540e-01,  3.05528481e+00,  1.60208732e-01,
        2.20455841e-11, -1.47496585e+00,  1.47496585e+00,  1.03864400e-10]), {'nfev': 253, 'fjac': array([[ 1.06173145e-14, -2.22611158e-11,  9.62733747e-15,
         3.34883692e-15,  9.39515392e-15, -2.12079589e-14,
         5.88738628e-15, -4.0